**Importing all the things we need, but before that you need to download and install llama on your PC.**

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough

**Defining the model**

In [ ]:
model = ChatOllama(
    model = "llama3.2")

**Loading our file which we will use to build out RAG system**

In [ ]:
raw_doc = TextLoader("./main_data.txt").load()

**We're not going to use whole file when asking question so we want to split it, also that will give us huge benefit when we will build our vector database for finding similler thing in the whole file**  

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,
chunk_overlap = 20)
document = text_splitter.split_documents(raw_doc)

In [ ]:
len(document)

124

**We're gonna use embedding for finding similler word**

In [ ]:
oembed = OllamaEmbeddings(model = "nomic-embed-text")

**Building the vector database also using the embedding in i**t

In [ ]:
db = Chroma.from_documents(document,embedding = oembed)

**Building our chat template it's like how we want our answer to be**

In [ ]:
chat_template_ = """
Answer the question based on the following context:
{context}
question: {question}
"""
chat_template = ChatPromptTemplate.from_template(chat_template_)

**To retrive relavent chunk (split sentances that we made earlier) from the question. We make our retriver with the help of vector database that we build earlier**

In [ ]:
retriver = db.as_retriever()

**We will make a function that will join all the chunk it(retriver) found similler**

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

**Making our chain or steps that the user input wiil go through**

In [ ]:
chain = ({"context":retriver|format_docs,"question":RunnablePassthrough()}|chat_template|model)

**Finally we use this chain to get our answer**

In [ ]:
print(chain.invoke("what is cell nucleus").content)

According to the context, the cell nucleus is:

* The most conspicuous organelle found in a eukaryotic cell.
* The information center of the cell.
* The place where the cell's chromosomes are housed.
